In [17]:
from sklearn.feature_selection import mutual_info_regression
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.express as px
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report)
from sklearn.exceptions import NotFittedError

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier,
                              GradientBoostingClassifier, BaggingClassifier, HistGradientBoostingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

In [18]:
data= pd.read_csv('/kaggle/input/framingham/framingham.csv')
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,male,39,4.0,No,0.0,0.0,0,0,No,195.0,106.0,70.0,26.97,80.0,77.0,0
1,female,46,2.0,No,0.0,0.0,0,0,No,250.0,121.0,81.0,28.73,95.0,76.0,0
2,male,48,1.0,Yes,20.0,0.0,0,0,No,245.0,127.5,80.0,25.34,75.0,70.0,0
3,female,61,3.0,Yes,30.0,0.0,0,1,No,225.0,150.0,95.0,28.58,65.0,103.0,1
4,female,46,3.0,Yes,23.0,0.0,0,0,No,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,female,48,2.0,Yes,20.0,NaN,0,0,No,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,female,44,1.0,Yes,15.0,0.0,0,0,No,210.0,126.5,87.0,19.16,86.0,NaN,0
4237,female,52,2.0,No,0.0,0.0,0,0,No,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,male,40,3.0,No,0.0,0.0,0,1,No,185.0,141.0,98.0,25.60,67.0,72.0,0


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sex              4240 non-null   object 
 1   age              4240 non-null   int64  
 2   education        4135 non-null   float64
 3   currentSmoker    4240 non-null   object 
 4   cigsPerDay       4211 non-null   float64
 5   BPMeds           4187 non-null   float64
 6   prevalentStroke  4240 non-null   int64  
 7   prevalentHyp     4240 non-null   int64  
 8   diabetes         4240 non-null   object 
 9   totChol          4190 non-null   float64
 10  sysBP            4240 non-null   float64
 11  diaBP            4240 non-null   float64
 12  BMI              4221 non-null   float64
 13  heartRate        4239 non-null   float64
 14  glucose          3852 non-null   float64
 15  TenYearCHD       4240 non-null   int64  
dtypes: float64(9), int64(4), object(3)
memory usage: 530.1+ KB


In [20]:
data.isnull().sum()

Sex                  0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [21]:
round(data.describe().T,2)

,count,mean,std,min,25%,50%,75%,max
age,4240.0,49.58,8.57,32.00,42.00,49.0,56.00,70.0
education,4135.0,1.98,1.02,1.00,1.00,2.0,3.00,4.0
cigsPerDay,4211.0,9.01,11.92,0.00,0.00,0.0,20.00,70.0
BPMeds,4187.0,0.03,0.17,0.00,0.00,0.0,0.00,1.0
prevalentStroke,4240.0,0.01,0.08,0.00,0.00,0.0,0.00,1.0
prevalentHyp,4240.0,0.31,0.46,0.00,0.00,0.0,1.00,1.0
totChol,4190.0,236.70,44.59,107.00,206.00,234.0,263.00,696.0
sysBP,4240.0,132.35,22.03,83.50,117.00,128.0,144.00,295.0
diaBP,4240.0,82.90,11.91,48.00,75.00,82.0,90.00,142.5
BMI,4221.0,25.80,4.08,15.54,23.07,25.4,28.04,56.8


In [22]:
# data.duplicated().sum()

In [23]:
# from ydata_profiling import ProfileReport
# profile=ProfileReport(data,explorative=True,dark_mode=True)
# profile

In [24]:
# data.columns

In [25]:
unique_values= {}
for i in data.columns:
    unique_values[i]= data[i].nunique()
    
pd.DataFrame(unique_values,index= ['unique_value']).T

,unique_value
Sex,2
age,39
education,4
currentSmoker,2
cigsPerDay,33
BPMeds,2
prevalentStroke,2
prevalentHyp,2
diabetes,2
totChol,248


In [26]:
print(data['Sex'].unique())

['male' 'female']


In [27]:
data['Sex'] = data['Sex'].map({'male': 1, 'female': 0})
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,No,0.0,0.0,0,0,No,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,No,0.0,0.0,0,0,No,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,Yes,20.0,0.0,0,0,No,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,Yes,30.0,0.0,0,1,No,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,Yes,23.0,0.0,0,0,No,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,Yes,20.0,NaN,0,0,No,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,Yes,15.0,0.0,0,0,No,210.0,126.5,87.0,19.16,86.0,NaN,0
4237,0,52,2.0,No,0.0,0.0,0,0,No,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,1,40,3.0,No,0.0,0.0,0,1,No,185.0,141.0,98.0,25.60,67.0,72.0,0


In [28]:
data['currentSmoker']= data['currentSmoker'].map({'Yes':1, 'No':0})
data['diabetes']= data['diabetes'].map({'Yes':1, 'No':0})

In [29]:
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN,0
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,1,40,3.0,0,0.0,0.0,0,1,0,185.0,141.0,98.0,25.60,67.0,72.0,0


In [30]:
# from sklearn import preprocessing
# from collections import defaultdict
# d = defaultdict(preprocessing.LabelEncoder)
# data['Sex'] = d['Sex'].fit_transform(data['Sex'])
# data

In [31]:
data.corr()['TenYearCHD'].sort_values(ascending= False)

TenYearCHD         1.000000
age                0.225408
sysBP              0.216374
prevalentHyp       0.177458
diaBP              0.145112
glucose            0.125590
diabetes           0.097344
Sex                0.088374
BPMeds             0.087519
totChol            0.082369
BMI                0.075300
prevalentStroke    0.061823
cigsPerDay         0.057755
heartRate          0.022907
currentSmoker      0.019448
education         -0.054248
Name: TenYearCHD, dtype: float64

# all undersampling at once

In [32]:
data.fillna(data.std(), inplace=True)
print(data.isna().sum())

Sex                0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64


In [34]:
# data.drop(columns=['education'],inplace=True)
# data
target= data['TenYearCHD']
columns_to_drop = ['education','TenYearCHD']
features = data.drop(columns_to_drop, axis=1)
features

,Sex,age,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1,39,0,0.0,0.000000,0,0,0,195.0,106.0,70.0,26.97,80.0,77.000000
1,0,46,0,0.0,0.000000,0,0,0,250.0,121.0,81.0,28.73,95.0,76.000000
2,1,48,1,20.0,0.000000,0,0,0,245.0,127.5,80.0,25.34,75.0,70.000000
3,0,61,1,30.0,0.000000,0,1,0,225.0,150.0,95.0,28.58,65.0,103.000000
4,0,46,1,23.0,0.000000,0,0,0,285.0,130.0,84.0,23.10,85.0,85.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,1,20.0,0.169544,0,0,0,248.0,131.0,72.0,22.00,84.0,86.000000
4236,0,44,1,15.0,0.000000,0,0,0,210.0,126.5,87.0,19.16,86.0,23.954335
4237,0,52,0,0.0,0.000000,0,0,0,269.0,133.5,83.0,21.47,80.0,107.000000
4238,1,40,0,0.0,0.000000,0,1,0,185.0,141.0,98.0,25.60,67.0,72.000000


In [35]:
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, ClusterCentroids, NearMiss, EditedNearestNeighbours

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

rus = RandomUnderSampler()
X_RUS, y_RUS = rus.fit_resample(X_train, y_train)

rf_model_rus = RandomForestClassifier(random_state=42)
rf_model_rus.fit(X_RUS, y_RUS)
y_pred_rus = rf_model_rus.predict(X_test)
print("Random Undersampling Accuracy:", accuracy_score(y_test, y_pred_rus))
print(classification_report(y_test, y_pred_rus))


tl = TomekLinks()
X_TL, y_TL = tl.fit_resample(X_train, y_train)

rf_model_tl = RandomForestClassifier(random_state=42)
rf_model_tl.fit(X_TL, y_TL)
y_pred_tl = rf_model_tl.predict(X_test)
print("\nTomek Links Accuracy:", accuracy_score(y_test, y_pred_tl))
print(classification_report(y_test, y_pred_tl))


cc = ClusterCentroids()
X_CC, y_CC = cc.fit_resample(X_train, y_train)

rf_model_cc = RandomForestClassifier(random_state=42)
rf_model_cc.fit(X_CC, y_CC)
y_pred_cc = rf_model_cc.predict(X_test)
print("\nCluster Centroids Accuracy:", accuracy_score(y_test, y_pred_cc))
print(classification_report(y_test, y_pred_cc))


nm = NearMiss()
X_NM, y_NM = nm.fit_resample(X_train, y_train)

rf_model_nm = RandomForestClassifier(random_state=42)
rf_model_nm.fit(X_NM, y_NM)
y_pred_nm = rf_model_nm.predict(X_test)
print("\nNearMiss Accuracy:", accuracy_score(y_test, y_pred_nm))
print(classification_report(y_test, y_pred_nm))


enn = EditedNearestNeighbours()
X_ENN, y_ENN = enn.fit_resample(X_train, y_train)

rf_model_enn = RandomForestClassifier(random_state=42)
rf_model_enn.fit(X_ENN, y_ENN)
y_pred_enn = rf_model_enn.predict(X_test)
print("\nEdited Nearest Neighbours Accuracy:", accuracy_score(y_test, y_pred_enn))
print(classification_report(y_test, y_pred_enn))

Random Undersampling Accuracy: 0.6470125786163522
              precision    recall  f1-score   support

           0       0.90      0.65      0.76      1077
           1       0.24      0.62      0.35       195

    accuracy                           0.65      1272
   macro avg       0.57      0.63      0.55      1272
weighted avg       0.80      0.65      0.70      1272


Tomek Links Accuracy: 0.8514150943396226
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      1077
           1       0.58      0.11      0.19       195

    accuracy                           0.85      1272
   macro avg       0.72      0.55      0.55      1272
weighted avg       0.82      0.85      0.81      1272



/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



Cluster Centroids Accuracy: 0.3411949685534591
              precision    recall  f1-score   support

           0       0.87      0.26      0.40      1077
           1       0.16      0.79      0.27       195

    accuracy                           0.34      1272
   macro avg       0.52      0.53      0.33      1272
weighted avg       0.77      0.34      0.38      1272


NearMiss Accuracy: 0.4111635220125786
              precision    recall  f1-score   support

           0       0.90      0.34      0.50      1077
           1       0.18      0.78      0.29       195

    accuracy                           0.41      1272
   macro avg       0.54      0.56      0.39      1272
weighted avg       0.79      0.41      0.47      1272


Edited Nearest Neighbours Accuracy: 0.8128930817610063
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      1077
           1       0.35      0.26      0.30       195

    accuracy                           

# Lots of empty value fill up with 0

In [36]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import TomekLinks

target = data['TenYearCHD']
columns_to_drop = ['TenYearCHD','education']
features = data.drop(columns_to_drop, axis=1)
x= features.values
y= target.values

tomeklinks= TomekLinks()
oversample= SMOTE()
steps= [("TL", tomeklinks), ("os", oversample)]
pipeline= Pipeline(steps=steps)

x_s, y_s = pipeline.fit_resample(x, y)

x_train, x_test,y_train, y_test = train_test_split(x_s, y_s, test_size=0.2, random_state=42)

model= RandomForestClassifier(random_state=42)
model.fit(x_train,y_train)

y_pred = model.predict(x_test)
print("Classification Report:\n",classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.93      0.90       693
           1       0.92      0.87      0.89       663

    accuracy                           0.90      1356
   macro avg       0.90      0.90      0.90      1356
weighted avg       0.90      0.90      0.90      1356



In [37]:
exc= ExtraTreesClassifier(criterion='entropy', n_estimators= 100, max_depth= 500, min_samples_split= 2, max_features='log2')
exc.fit(x_train,y_train)
y_pred= exc.predict(x_test)
print("classification report: \n", classification_report(y_test, y_pred))

classification report: 
               precision    recall  f1-score   support

           0       0.89      0.94      0.92       693
           1       0.94      0.88      0.91       663

    accuracy                           0.91      1356
   macro avg       0.92      0.91      0.91      1356
weighted avg       0.92      0.91      0.91      1356



In [38]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
df= pd.DataFrame(data)
imputer= IterativeImputer(max_iter=10, random_state=42)
impute_data= imputer.fit_transform(df)
data= pd.DataFrame(impute_data, columns= df.columns)

data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1.0,39.0,4.0,0.0,0.0,0.000000,0.0,0.0,0.0,195.0,106.0,70.0,26.97,80.0,77.000000,0.0
1,0.0,46.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.0,250.0,121.0,81.0,28.73,95.0,76.000000,0.0
2,1.0,48.0,1.0,1.0,20.0,0.000000,0.0,0.0,0.0,245.0,127.5,80.0,25.34,75.0,70.000000,0.0
3,0.0,61.0,3.0,1.0,30.0,0.000000,0.0,1.0,0.0,225.0,150.0,95.0,28.58,65.0,103.000000,1.0
4,0.0,46.0,3.0,1.0,23.0,0.000000,0.0,0.0,0.0,285.0,130.0,84.0,23.10,85.0,85.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0.0,48.0,2.0,1.0,20.0,0.169544,0.0,0.0,0.0,248.0,131.0,72.0,22.00,84.0,86.000000,0.0
4236,0.0,44.0,1.0,1.0,15.0,0.000000,0.0,0.0,0.0,210.0,126.5,87.0,19.16,86.0,23.954335,0.0
4237,0.0,52.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.0,269.0,133.5,83.0,21.47,80.0,107.000000,0.0
4238,1.0,40.0,3.0,0.0,0.0,0.000000,0.0,1.0,0.0,185.0,141.0,98.0,25.60,67.0,72.000000,0.0


In [ ]:
#data.dropna(inplace=True) #0.8401639344262295
# data.fillna(1, inplace=True) # lr=0.8561320754716981
# data

In [ ]:
# df= data.iloc[:4020]
# count_0=df['TenYearCHD'].eq(0).sum()
# count_1=df['TenYearCHD'].eq(1).sum()

# print("count 0",count_0)
# print("count 1",count_1)

In [ ]:
data.info()

In [ ]:
# data.astype(int)

In [ ]:
# num_rows = len(data.columns) + 2
# plt.figure(figsize=(9, num_rows * 3))
# for i, col in enumerate(data.columns):
#     plt.subplot(num_rows, 3, i + 1)
#     sns.histplot(data=data, x=col , multiple='stack')
#     plt.xlabel(col)
# plt.tight_layout()
# plt.show()

In [39]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.datasets import make_classification

x = features.values
y = target.values

xx_train, xx_test, yy_train, yy_test = train_test_split(x, y, test_size=0.2, random_state=42)


smote = SMOTE(random_state=42)
xx_train, yy_train = smote.fit_resample(xx_train, yy_train)

model = RandomForestClassifier(random_state=42)
model.fit(xx_train, yy_train)

yy_pred = model.predict(xx_test)

print("Classification Report:\n", classification_report(yy_test, yy_pred))

# Necessary imports
# from imblearn.over_sampling import SMOTE
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report


# # Convert features and target to numpy arrays (if they are not already)
# X = features.values
# y = target.values

# # Split the dataset
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Apply SMOTE to the training data
# smote = SMOTE(random_state=42)
# x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# # Train a RandomForestClassifier
# model = RandomForestClassifier(random_state=42)
# model.fit(x_train_resampled, y_train_resampled)

# # Make predictions
# y_pred = model.predict(x_test)

# # Print classification report
# print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.94      0.90       725
           1       0.30      0.15      0.20       123

    accuracy                           0.83       848
   macro avg       0.58      0.55      0.55       848
weighted avg       0.79      0.83      0.80       848



In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, cohen_kappa_score, confusion_matrix

def evaluate_model(model, xx_train, yy_train, xx_test, yy_test):
    # Predictions
    y_pred_train = model.predict(xx_train)
    y_pred_test = model.predict(xx_test)
    
    accuracy_train = accuracy_score(yy_train, y_pred_train)
    accuracy_test = accuracy_score(yy_test, y_pred_test)
    precision_test = precision_score(yy_test, y_pred_test)
    recall_test = recall_score(yy_test, y_pred_test)
    f1_test = f1_score(yy_test, y_pred_test)
    auc = roc_auc_score(yy_test, model.predict_proba(xx_test)[:, 1])
    mcc_test = matthews_corrcoef(yy_test, y_pred_test)
    kappa_test = cohen_kappa_score(yy_test, y_pred_test)

    tn, fp, fn, tp = confusion_matrix(yy_test, y_pred_test).ravel()
    specificity_test = tn / (tn + fp)
    sensitivity_test = recall_test  
    
    print(f"Train Accuracy: {accuracy_train*100:.2f}%")
    print(f"Test Accuracy: {accuracy_test*100:.2f}%")
    print(f"Precision: {precision_test*100:.2f}%")
    print(f"Recall/Sensitivity: {sensitivity_test*100:.2f}%")
    print(f"Specificity: {specificity_test*100:.2f}%")
    print(f"F1-Score: {f1_test*100:.2f}%")
    print(f"AUC-ROC: {auc*100:.2f}%")
    print(f"MCC: {mcc_test*100:.2f}%")
    print(f"Kappa: {kappa_test*100:.2f}%")

In [41]:
from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression(solver='newton-cg',multi_class='multinomial') #0.8561320754716981
#lr =LogisticRegression(solver='saga', multi_class='multinomial') #0.8561320754716981
lr = LogisticRegression(solver='liblinear',multi_class='ovr', C=0.1)
lr.fit(xx_train, yy_train)
evaluate_model(lr, xx_train, yy_train, xx_test, yy_test)

Train Accuracy: 67.52%
Test Accuracy: 63.92%
Precision: 22.85%
Recall/Sensitivity: 62.60%
Specificity: 64.14%
F1-Score: 33.48%
AUC-ROC: 69.23%
MCC: 19.24%
Kappa: 15.53%


In [42]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

sgd_clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=5000, random_state=500))
#sgd_clf.fit(x_train_resampled, y_train_resampled)  #fit(x_train, y_train)
sgd_clf.fit(xx_train, yy_train)

y_pred = sgd_clf.predict(xx_test)
accuracy = accuracy_score(yy_test, yy_pred)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

Test Accuracy: 82.55%


In [43]:
from sklearn.tree import DecisionTreeClassifier
#model = DecisionTreeClassifier(random_state=42).fit(x_train_resampled, y_train_resampled) 
model = DecisionTreeClassifier(random_state=42).fit(xx_train, yy_train)

train_preds = model.predict(xx_train)
print(train_preds)

[0 0 0 ... 1 1 1]


In [44]:
train_preds = model.predict(xx_train)
train_accuracy = accuracy_score(yy_train, train_preds)
print("Training Accuracy: {:.2f}%".format(train_accuracy * 100))
test_preds = model.predict(xx_test)
test_accuracy = accuracy_score(yy_test, test_preds)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

Training Accuracy: 100.00%
Test Accuracy: 74.53%


In [ ]:
model=DecisionTreeClassifier(max_depth=5, random_state=42)
#model.fit(x_train_resampled, y_train_resampled)
model.fit(xx_train, yy_train)

evaluate_model(model, xx_train, yy_train, xx_test, yy_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#model=RandomForestClassifier(n_jobs=-1,random_state=42,max_features=2,min_samples_split=80, min_samples_leaf=50,bootstrap=False)
model=RandomForestClassifier(class_weight='balanced')
model.fit(xx_train,yy_train)
evaluate_model(model, xx_train, yy_train, xx_test, yy_test)

In [ ]:
model=RandomForestClassifier(n_jobs=-1,random_state=42,n_estimators=10,max_depth=7,max_leaf_nodes=4,
                             max_features=32).fit(xx_train, yy_train)
evaluate_model(model, xx_train, yy_train, xx_test, yy_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=.05).fit(xx_train, yy_train)
evaluate_model(model, xx_train, yy_train, xx_test, yy_test)


In [ ]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss')
xgb_classifier.fit(xx_train, yy_train)

y_pred = xgb_classifier.predict(xx_test)
accuracy_xgb = accuracy_score(yy_test, y_pred)
print("XGBoost Accuracy: {:.2f}%".format(accuracy_xgb * 100))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model =KNeighborsClassifier(n_neighbors=20) 
model.fit(xx_train, yy_train)
evaluate_model(model, xx_train, yy_train, xx_test, yy_test)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model=AdaBoostClassifier(n_estimators=10, random_state=42) 
model.fit(xx_train, yy_train)
evaluate_model(model, xx_train, yy_train, xx_test, yy_test)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
exc= ExtraTreesClassifier(criterion='entropy', n_estimators= 100, max_depth= 500, min_samples_split= 2, max_features='log2')
exc.fit(xx_train,yy_train)
y_pred= exc.predict(xx_test)
print("classification report: \n", classification_report(yy_test, y_pred))

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, xx_train, yy_train, cv=kf, scoring='accuracy')
print(f"{scores}")
print(f"Mean Accuracy: {scores.mean():.2f}")
print(f"Standard Deviation: {scores.std():.2f}")

In [ ]:
rf_model = RandomForestClassifier(n_estimators=1)
cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'),x_train, y_train,cv=3)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(features)
print(scaled.shape)
scaled

# Using Gen = Genarative Adversial Network
deep-learning er oversampling

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

The Generator takes random noise as input and generates data:

In [ ]:
def build_generator(noise_dim):
    model = tf.keras.Sequential()
    
#     model.add(layers.Dense(16, input_dim= noise_dim))
#     model.add(layers.LeakyReLU(alpha=0.2))
#     model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(32, input_dim= noise_dim),)
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(64))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    
    #model.add(layers.Dense(128, input_dim=noise_dim))
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(np.prod(features.shape[1:]), activation='tanh'))
    model.add(layers.Reshape(features.shape[1:]))
    
    return model

The Discriminator distinguishes between real and fake data:

In [ ]:
def build_discriminator(data_shape):
    model= tf.keras.Sequential()
    model.add(layers.Flatten(input_shape= data_shape))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha= 0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha= 0.2))
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha= 0.2))
    
    model.add(layers.Dense(64))
    model.add(layers.LeakyReLU(alpha= 0.2))
    model.add(layers.Dense(32))
    model.add(layers.LeakyReLU(alpha= 0.2))
    
    model.add(layers.Dense(1, activation= 'sigmoid'))
    
    return model

In [ ]:
noise_dim= 100
generator= build_generator(noise_dim)
discriminator= build_discriminator(features.shape[1:])
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# GAN model
discriminator.trainable = False
gan_input = layers.Input(shape=(noise_dim,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)

gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

The training process involves training the Discriminator and the Generator

Symmetry around 0 allows the generator to learn from both positive and negative values.
A standard deviation of 1 provides a good range for the noise, giving enough variation for the generator to create diverse samples.

np.ones((batch_size, 1)): এখানে আসল ডেটার জন্য লেবেল বা লক্ষ্য মান 1 হিসাবে দেওয়া হচ্ছে। অর্থাৎ, ডিসক্রিমিনেটরকে শেখানো হচ্ছে যে এই ডেটাগুলি আসল।

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras import backend as K

# def build_discriminator(input_shape):
#     model = Sequential()
#     model.add(Dense(512, input_dim=input_shape))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dropout(0.4))
#     model.add(Dense(256))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dropout(0.4))
#     model.add(Dense(1, activation='sigmoid'))
#     return model

# def build_generator(noise_dim):
#     model = Sequential()
#     model.add(Dense(256, input_dim=noise_dim))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(BatchNormalization(momentum=0.8))
#     model.add(Dense(512))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(BatchNormalization(momentum=0.8))
#     model.add(Dense(1024))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(BatchNormalization(momentum=0.8))
#     model.add(Dense(features.shape[1], activation='tanh')) 
#     return model

# def build_gan(generator, discriminator):
#     model = Sequential()
#     model.add(generator)
#     discriminator.trainable = False
#     model.add(discriminator)
#     return model

# def train_gan(gan, generator, discriminator, features, target, noise_dim, epochs=100, batch_size=64, label_smoothing=0.9, d_steps=2):
#     discriminator_optimizer = Adam(learning_rate=0.0004, beta_1=0.5)
#     generator_optimizer = Adam(learning_rate=0.0001, beta_1=0.5)
    
#     discriminator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer, metrics=['accuracy'])
#     gan.compile(loss='binary_crossentropy', optimizer=generator_optimizer)

#     for epoch in range(epochs):
#         for _ in range(d_steps):
#             idx = np.random.randint(0, features.shape[0], batch_size)
#             real_data = features.iloc[idx].values
#             real_labels = target.iloc[idx].values.reshape(-1, 1)
#             real_labels = real_labels * label_smoothing

#             noise = np.random.normal(0, 1, (batch_size, noise_dim))
#             generated_data = generator.predict(noise)
#             fake_labels = np.random.uniform(0.0, 0.1, (batch_size, 1))

#             d_loss_real = discriminator.train_on_batch(real_data, real_labels)
#             d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)

#             d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

#         noise = np.random.normal(0, 1, (batch_size, noise_dim))
#         valid_y = np.ones((batch_size, 1))
#         g_loss = gan.train_on_batch(noise, valid_y)

#         if epoch % 100 == 0:
#             print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss}]")

# # Example usage
# noise_dim = 100
# epochs = 100
# batch_size = 64

# discriminator = build_discriminator(features.shape[1])
# generator = build_generator(noise_dim)
# gan = build_gan(generator, discriminator)

# train_gan(gan, generator, discriminator, features, target, noise_dim, epochs=epochs, batch_size=batch_size, label_smoothing=0.8, d_steps=2)

In [ ]:
# def train_gan(gan, generator, discriminator, data, noise_dim, epochs=100, batch_size=64):
#     for epoch in range(epochs):
#         # Train the discriminator
#         idx = np.random.randint(0, data.shape[0], batch_size)
#         real_data = data.iloc[idx]  # Use .iloc for row indexing
        
#         noise = np.random.normal(0, 1, (batch_size, noise_dim))
#         generated_data = generator.predict(noise)
        
#         d_loss_real = discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
#         d_loss_fake = discriminator.train_on_batch(generated_data, np.zeros((batch_size, 1)))
        
#         d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
#         # Train the generator
#         noise = np.random.normal(0, 1, (batch_size, noise_dim))
#         valid_y = np.array([1] * batch_size)
#         g_loss = gan.train_on_batch(noise, valid_y)

#         if epoch % 100 == 0:
#             print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]:.2f}] [G loss: {g_loss}]")

In [ ]:
# def train_gan(gan, generator, discriminator, features, noise_dim, epochs=100, batch_size=64):
#     for epoch in range(epochs):
#         # Train the discriminator
#         idx = np.random.randint(0, features.shape[0], batch_size)
#         real_data = features.iloc[idx]  # Use .iloc for integer-based indexing
        
#         noise = np.random.normal(0, 1, (batch_size, noise_dim))
#         generated_data = generator.predict(noise)
        
#         d_loss_real = discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
#         d_loss_fake = discriminator.train_on_batch(generated_data, np.zeros((batch_size, 1)))
        
#         d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
#         # Train the generator
#         noise = np.random.normal(0, 1, (batch_size, noise_dim))
#         valid_y = np.array([1] * batch_size)
#         g_loss = gan.train_on_batch(noise, valid_y)

#         if epoch % 100 == 0:
#             print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]:.2f}] [G loss: {g_loss}]")

# train_gan(gan, generator, discriminator, features, noise_dim, epochs=100, batch_size=64)

In [ ]:
def train_gan(gan, generator, discriminator, features, target, noise_dim, epochs=100, batch_size=64):
    for epoch in range(epochs):
        # Train & discriminator on real data
        idx= np.random.randint(0, features.shape[0], batch_size)
        real_data= features.iloc[idx]  
        real_labels= target.iloc[idx].values.reshape(-1, 1) 

        # Generate data
        noise= np.random.normal(0, 1, (batch_size, noise_dim))
        generated_data= generator.predict(noise)

        d_loss_real= discriminator.train_on_batch(real_data, real_labels)
        d_loss_fake= discriminator.train_on_batch(generated_data, np.zeros((batch_size, 1)))
        
        d_loss = 0.5* np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, noise_dim))
        valid_y = np.ones((batch_size, 1))  # Generator aims to create "real" data, so label it as 1
        g_loss = gan.train_on_batch(noise, valid_y)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss}]")
train_gan(gan, generator, discriminator, features, target,noise_dim, epochs=100, batch_size=64)

In [ ]:
def generate_synthetic_data(generator, noise_dim, num_zeros, num_ones):
    noise_for_zeros = np.random.normal(0, 1, (num_zeros, noise_dim))
    generated_zeros = generator.predict(noise_for_zeros)

    noise_for_ones = np.random.normal(0, 1, (num_ones, noise_dim))
    generated_ones = generator.predict(noise_for_ones)

    generated_data = np.concatenate([generated_zeros, generated_ones], axis=0)
    
    generated_labels = np.concatenate([np.zeros((num_zeros, 1)), np.ones((num_ones, 1))], axis=0)
    return generated_data, generated_labels

num_generated_zeros = 10
num_generated_ones = 4000  

generated_data, generated_labels = generate_synthetic_data(generator, noise_dim, num_generated_zeros, num_generated_ones)
combined_data = np.concatenate([features, generated_data], axis=0)
combined_targets = np.concatenate([target.values.reshape(-1, 1), generated_labels], axis=0)

print(f"Generated {num_generated_zeros} samples with label 0 and {num_generated_ones} samples with label 1.")

In [ ]:
# num_generated_samples = 10000

# # Generate synthetic data
# noise = np.random.normal(0, 1, (num_generated_samples, noise_dim))
# generated_data = generator.predict(noise)

# combined_data = np.concatenate([features, generated_data], axis=0)
# combined_targets = np.concatenate([target, np.ones(num_generated_samples)], axis=0)  #synthetic data targets are 1

In [ ]:
count_zeros = np.sum(combined_targets == 0)
count_ones = np.sum(combined_targets == 1)

print(f"Count of 0s (No Heart Attack): {count_zeros}")
print(f"Count of 1s (Heart Attack): {count_ones}")

In [ ]:
# from sklearn.manifold import TSNE # Apply t-SNE to reduce dimensions to 2D
# import matplotlib.pyplot as plt

# tsne = TSNE(n_components=2, random_state=42)
# combined_data_2d = tsne.fit_transform(combined_data)

# plt.figure(figsize=(8, 6))
# plt.scatter(combined_data_2d[:len(data), 0], combined_data_2d[:len(data), 1], c='blue', label='Real Data')
# plt.scatter(combined_data_2d[len(data):, 0], combined_data_2d[len(data):, 1], c='red', label='Synthetic Data')
# plt.legend()
# plt.title("Real and Synthetic Data")
# plt.show()

In [ ]:
# from sklearn.decomposition import PCA #Apply PCA to reduce dimensions to 2D
# import matplotlib.pyplot as plt 
# pca = PCA(n_components=2)
# combined_data_2d = pca.fit_transform(combined_data)

# plt.figure(figsize=(8, 6))
# plt.scatter(combined_data_2d[:len(data), 0], combined_data_2d[:len(data), 1], c='blue', label='Real Data')
# plt.scatter(combined_data_2d[len(data):, 0], combined_data_2d[len(data):, 1], c='red', label='Synthetic Data')
# plt.legend()
# plt.title("PCA Visualization of Real and Synthetic Data")
# plt.show()

# if the data length and the combinded data length size are not similer

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# import numpy as np

# num_generated_samples = 2700
# noise_dim = 100  
# features = np.random.rand(5000, 14)  # Replace with actual features
# target = np.random.randint(0, 2, size=(5000,))  # Replace with actual target

# noise = np.random.normal(0, 1, (num_generated_samples, noise_dim))
# generated_data = generator.predict(noise)  # Ensure the generator's output shape matches the features shape

# combined_data = np.concatenate([features, generated_data], axis=0)
# combined_targets = np.concatenate([target, np.ones(num_generated_samples)], axis=0)

# # Ensure that the lengths match
# if len(combined_data) != len(combined_targets):
#     min_length = min(len(combined_data), len(combined_targets))
#     combined_data = combined_data[:min_length]
#     combined_targets = combined_targets[:min_length]

# scaler = MinMaxScaler(feature_range=(-1, 1))
# scaled_combined_data = scaler.fit_transform(combined_data)

# x_train, x_test, y_train, y_test = train_test_split(scaled_combined_data, combined_targets, test_size=0.2, random_state=42)

# print('Train shape:', x_train.shape)
# print('Test shape:', x_test.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(combined_data, combined_targets, test_size=0.2, random_state=42)

print('Train shape:', x_train.shape)
print('Test shape:', x_test.shape)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
exc= ExtraTreesClassifier(criterion='entropy', n_estimators= 100, max_depth= 500, min_samples_split= 2, max_features='log2')
exc.fit(x_train,y_train.ravel())
y_pred= exc.predict(x_test)
print("classification report: \n", classification_report(y_test, y_pred))

In [ ]:
classifiers = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(random_state=42),
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),
    #"MultinomialNB": MultinomialNB(),
    "KNeighbors": KNeighborsClassifier(),
    #"RadiusNeighbors": RadiusNeighborsClassifier(),
    "SVC": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "MLPClassifier": MLPClassifier(random_state=42)
}
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall', 'Specificity', 'ROC-AUC', 'Confusion Matrix'])


def evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test):
    global results_df
    for name, clf in classifiers.items():
        try:
            clf.fit(x_train, y_train) 
            y_pred = clf.predict(x_test)  

            cm = confusion_matrix(y_test, y_pred)
            if len(cm.ravel()) == 4:  
                tn, fp, fn, tp = cm.ravel()
                specificity = tn / (tn + fp) 
            else:
                specificity = None

            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            report = classification_report(y_test, y_pred, output_dict=True)
            precision = report['weighted avg']['precision']
            recall = report['weighted avg']['recall']

            if hasattr(clf, "predict_proba"): 
                y_proba = clf.predict_proba(x_test)[:, 1]
                roc_auc = roc_auc_score(y_test, y_proba)
            else:
                roc_auc = None
                
            new_row = pd.DataFrame({
                'Model': [name],
                'Accuracy': [acc],
                'F1 Score': [f1],
                'Precision': [precision],
                'Recall': [recall],
                'Specificity': [specificity],
                'ROC-AUC': [roc_auc],
                'Confusion Matrix': [cm]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

        except (ValueError, NotFittedError) as e:
            print(f"Error with classifier {name}: {e}")
            
evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test)
print(results_df)

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(scaled, target, test_size=0.2,random_state=42)

# print('Train shape:', x_train.shape)
# print('Test shape:', x_test.shape)

In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

lgbm = lgb.LGBMClassifier(
    boosting_type='dart',
    learning_rate=0.05,
    n_estimators=40,
    num_leaves=31,
    random_state=42
)
lgbm.fit(x_train, y_train)

y_pred = lgbm.predict(x_test)
y_prob = lgbm.predict_proba(x_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC AUC Score: {roc_auc:.4f}')

In [ ]:
# import lightgbm as lgb
# import pandas as pd
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# lgbm = lgb.LGBMClassifier(random_state=42)

# param_grid = {
#     'boosting_type': ['gbdt', 'dart', 'rf'],
#     'num_leaves': [31, 50, 100],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'n_estimators': [20, 40, 60, 80, 100]
# }

# grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=1)

# grid_search.fit(x_train, y_train)

# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# print(f'Best Parameters: {best_params}')

# y_pred = best_model.predict(x_test)
# y_prob = best_model.predict_proba(x_test)[:, 1]

# functional model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


target = data['TenYearCHD']
columns_to_drop = ['TenYearCHD','education']
features = data.drop(columns_to_drop, axis=1)
x = features.values
y = target.values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

optimizer = Adam(learning_rate=0.02)

def build_model(input_dim):
    inputs = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x) 
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_dim = x_train_scaled.shape[1]
model = build_model(input_dim)

early_stopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
history = model.fit(x_train_scaled, y_train, 
                    epochs=100, 
                    batch_size=32, 
                    validation_split=0.2, 
                    callbacks=[early_stopping],
                    verbose=1)

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test_scaled, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")
y_pred = (model.predict(x_test_scaled) > 0.5).astype("int32") 
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Sequential Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

def build_sequential_model(input_dim, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid')) 
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

input_dim = x_train_scaled.shape[1]
model = build_sequential_model(input_dim)

history = model.fit(
    x_train_scaled, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    validation_split=0.2, 
    callbacks=[early_stopping],
    verbose=1
)
test_loss, test_accuracy = model.evaluate(x_test_scaled, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, cohen_kappa_score, confusion_matrix

# def evaluate_model(model, x_train, y_train, x_test, y_test):
#     # Predictions
#     y_pred_train = model.predict(x_train)
#     y_pred_test = model.predict(x_test)
    
#     accuracy_train= accuracy_score(y_train, y_pred_train)
#     accuracy_test= accuracy_score(y_test, y_pred_test)
#     precision_test= precision_score(y_test, y_pred_test)
#     recall_test= recall_score(y_test, y_pred_test)
#     f1_test= f1_score(y_test, y_pred_test)
#     auc= roc_auc_score(y_test, model.predict_proba(x_test)[:, 1])
#     mcc_test= matthews_corrcoef(y_test, y_pred_test)
#     kappa_test= cohen_kappa_score(y_test, y_pred_test)

#     tn, fp, fn, tp= confusion_matrix(y_test, y_pred_test).ravel()
#     specificity_test= tn / (tn + fp)
#     sensitivity_test= recall_test  
    
#     print(f"Train Accuracy: {accuracy_train*100:.2f}%")
#     print(f"Test Accuracy: {accuracy_test*100:.2f}%")
#     print(f"Precision: {precision_test*100:.2f}%")
#     print(f"Recall/Sensitivity: {sensitivity_test*100:.2f}%")
#     print(f"Specificity: {specificity_test*100:.2f}%")
#     print(f"F1-Score: {f1_test*100:.2f}%")
#     print(f"AUC-ROC: {auc*100:.2f}%")
#     print(f"MCC: {mcc_test*100:.2f}%")
#     print(f"Kappa: {kappa_test*100:.2f}%")

In [ ]:
from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression(solver='newton-cg',multi_class='multinomial') #0.8561320754716981
#lr =LogisticRegression(solver='saga', multi_class='multinomial') #0.8561320754716981
lr = LogisticRegression(solver='liblinear',multi_class='ovr', C=0.1)
#lr.fit(x_train_resampled, y_train_resampled) #fit(x_train, y_train)
lr.fit(x_train, y_train)
evaluate_model(lr, x_train, y_train, x_test, y_test)

# import shap
# explainer = shap.Explainer(lr, x_train)
# shap_values = explainer.shap_values(x_test[:100])
# plt.figure(figsize=(16, 12))
# shap.summary_plot(shap_values, features=x_test[:100], feature_names=features.columns)
# plt.show()

In [ ]:
# from imblearn.over_sampling import SMOTE
# from sklearn.linear_model import LogisticRegression

# smote = SMOTE(random_state=42)
# x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# lr = LogisticRegression(solver='liblinear', multi_class='ovr')
# lr.fit(x_train_resampled, y_train_resampled)

# accuracy = lr.score(x_test, y_test) * 100
# print(f"Accuracy: {accuracy:.2f}%")

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler
# #rus = RandomUnderSampler(sampling_strategy=0.4, random_state=42) #drop 0% data
# rus = RandomUnderSampler(random_state=42)
# x_train_resampled, y_train_resampled = rus.fit_resample(x_train, y_train)

In [ ]:
# x_train_resampled_series = pd.Series(y_train_resampled, name='TenYearCHD')

# class_counts = x_train_resampled_series.value_counts()

# count_0 = class_counts.get(0, 0) 
# count_1 = class_counts.get(1, 0)  
# print(f"{count_0}")
# print(f"{count_1}")

# SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

sgd_clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=5000, random_state=500))
#sgd_clf.fit(x_train_resampled, y_train_resampled)  #fit(x_train, y_train)
sgd_clf.fit(x_train, y_train)

y_pred = sgd_clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

# DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
#model = DecisionTreeClassifier(random_state=42).fit(x_train_resampled, y_train_resampled) 
model = DecisionTreeClassifier(random_state=42).fit(x_train, y_train)

train_preds = model.predict(x_train)
print(train_preds)

In [ ]:
train_preds = model.predict(x_train)
train_accuracy = accuracy_score(y_train, train_preds)
print("Training Accuracy: {:.2f}%".format(train_accuracy * 100))
test_preds = model.predict(x_test)
test_accuracy = accuracy_score(y_test, test_preds)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

In [ ]:
model=DecisionTreeClassifier(max_depth=7, random_state=42)
#model.fit(x_train_resampled, y_train_resampled)
model.fit(x_train, y_train)

evaluate_model(model, x_train, y_train, x_test, y_test)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#model=RandomForestClassifier(n_jobs=-1,random_state=42,max_features=2,min_samples_split=80, min_samples_leaf=50,bootstrap=False)
model=RandomForestClassifier(class_weight='balanced')
model.fit(x_train,y_train)
model.fit(x_train,y_train)
evaluate_model(model, x_train, y_train, x_test, y_test)

In [ ]:
model=RandomForestClassifier(n_jobs=-1,random_state=42,n_estimators=5,max_depth=7,max_leaf_nodes=4,
                             max_features=32).fit(x_train, y_train)
evaluate_model(model, x_train, y_train, x_test, y_test)

# KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model =KNeighborsClassifier(n_neighbors=20) 
model.fit(x_train, y_train)
evaluate_model(model, x_train, y_train, x_test, y_test)

# AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model=AdaBoostClassifier(n_estimators=10, random_state=42) 
model.fit(x_train, y_train)
evaluate_model(model, x_train, y_train, x_test, y_test)

# xgBoosting

In [ ]:
import xgboost as xgb

xgb_classifier = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss')
xgb_classifier.fit(x_train, y_train)

y_pred = xgb_classifier.predict(x_test)
accuracy_xgb = accuracy_score(y_test, y_pred)
print("XGBoost Test Accuracy: {:.2f}%".format(accuracy_xgb * 100))

# Gradient Boosting Classifire

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=.02).fit(x_train,y_train)
evaluate_model(model, x_train, y_train, x_test, y_test)

# Cross Validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, x_train, y_train, cv=kf, scoring='accuracy')

print(f"{scores}")
print(f"Mean Accuracy: {scores.mean():.2f}")
print(f"Standard Deviation: {scores.std():.2f}")

In [ ]:
rf_model = RandomForestClassifier(n_estimators=1)
cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'),x_train, y_train,cv=3)

# fully conected Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

input_shape= (14,)
inputs= Input(shape= input_shape)

x= Dense(512, activation='relu')(inputs)
x= Dense(256, activation= 'relu')(x)
x= Dense(128, activation= 'relu')(x)
x= Dense(64, activation= 'relu')(x)
x= Dense(32, activation= 'relu')(x)
x= Dense(16, activation= 'relu')(x)
# x= Dense(4, activation= 'relu')(x)

output= Dense(1, activation= 'sigmoid')(x)

model= Model(inputs= inputs, outputs= output)

model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history= model.fit(
    x_train,
    y_train,
    epochs= 100,
    verbose= 1,
    batch_size= 32
    )

In [ ]:
from sklearn.preprocessing import StandardScaler

def predict_heart_attack(model, features, scaler=None):
    feature_names = [
        'Sex', 'age', 'currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke',
        'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose'
    ]
    
    # Convert features dictionary to DataFrame
    features_df = pd.DataFrame([features], columns=feature_names)
    if scaler:
        features_df = scaler.transform(features_df)

    prediction = model.predict(features_df)
    
    return prediction[0]

sgd_model = make_pipeline(StandardScaler(), SGDClassifier(max_iter=5000, random_state=500)).fit(x_train, y_train)
dt_model = DecisionTreeClassifier(max_depth=7, random_state=63).fit(x_train, y_train)

input_features = {
    'Sex': 1,               
    'age': 61,
    'currentSmoker': 1,     
    'cigsPerDay': 30,
    'BPMeds': 0,
    'prevalentStroke': 0,
    'prevalentHyp': 1,      
    'diabetes': 0,
    'totChol': 255,
    'sysBP': 150,
    'diaBP': 95,
    'BMI': 28.58,
    'heartRate': 65,
    'glucose': 103
}

sgd_result = predict_heart_attack(sgd_model, input_features, scaler=sgd_model.named_steps['standardscaler'])
print(f'SGDClassifier Predicted result: {"Heart Attack" if sgd_result == 1 else "No Heart Attack"}')

dt_result = predict_heart_attack(dt_model, input_features)
print(f'DecisionTreeClassifier Predicted result: {"Heart Attack" if dt_result == 1 else "No Heart Attack"}')